In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/analysis-of-chest-x-rays-indiana-university/__results__.html
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__resultx__.html
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__notebook__.ipynb
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__output__.json
/kaggle/input/analysis-of-chest-x-rays-indiana-university/custom.css
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__results___files/__results___72_0.png
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__results___files/__results___79_0.png
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__results___files/__results___61_0.png
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__results___files/__results___70_0.png
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__results___files/__results___63_0.png
/kaggle/input/analysis-of-chest-x-rays-indiana-university/__results___files/__results___59_0.png
/kaggle/input/analysis-of-chest-x-rays-indiana-u

In [2]:
!git clone https://github.com/openai/CLIP.git

Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256 (from 1)
Receiving objects: 100% (256/256), 8.93 MiB | 37.93 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [3]:
%cd CLIP


/kaggle/working/CLIP


In [4]:
%cd /kaggle/working/CLIP


/kaggle/working/CLIP


In [5]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-xiyc96nv
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-xiyc96nv
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import os
import pandas as pd
from PIL import Image
import torch
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 143MiB/s]


In [7]:
image_dir = "/kaggle/input/chest-xrays-indiana-university/images/images_normalized"
projection_csv = "/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv"
report_csv = "/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv"

In [8]:
proj_df = pd.read_csv(projection_csv)
report_df = pd.read_csv(report_csv)

# Assuming 'study_id' is the key to join
merged_df = proj_df.merge(report_df, on="uid", how="inner")
print(merged_df.columns)
print(merged_df.head())


Index(['uid', 'filename', 'projection', 'MeSH', 'Problems', 'image',
       'indication', 'comparison', 'findings', 'impression'],
      dtype='object')
   uid                filename projection  \
0    1  1_IM-0001-4001.dcm.png    Frontal   
1    1  1_IM-0001-3001.dcm.png    Lateral   
2    2  2_IM-0652-1001.dcm.png    Frontal   
3    2  2_IM-0652-2001.dcm.png    Lateral   
4    3  3_IM-1384-1001.dcm.png    Frontal   

                                                MeSH  \
0                                             normal   
1                                             normal   
2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
3  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
4                                             normal   

                        Problems                                image  \
0                         normal            Xray Chest PA and Lateral   
1                         normal            Xray Chest PA and Lateral   
2  Cardiomegaly;Pulm

In [9]:

print("Projections CSV columns:", proj_df.columns.tolist())
print("Reports CSV columns:", report_df.columns.tolist())



Projections CSV columns: ['uid', 'filename', 'projection']
Reports CSV columns: ['uid', 'MeSH', 'Problems', 'image', 'indication', 'comparison', 'findings', 'impression']


In [10]:
merged_df.to_csv("indiana_merged.csv", index=False)


In [11]:
from torch.utils.data import Dataset, DataLoader

class IndianaChestXrayDataset(Dataset):
    def __init__(self, dataframe, image_dir, preprocess, text_col="findings"):
        self.df = dataframe
        self.image_dir = image_dir
        self.preprocess = preprocess
        self.text_col = text_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row["filename"])

        # Load and preprocess image
        image = Image.open(image_path).convert("RGB")
        image = self.preprocess(image)

        # Select report text
        text = row[self.text_col]
        if pd.isnull(text):
            text = "No report"

        return image, text


In [12]:
train_dataset = IndianaChestXrayDataset(
    merged_df,
    image_dir=image_dir,
    preprocess=preprocess,
    text_col="findings"  # or "impression"
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)


In [13]:
import torch.nn as nn

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.2)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(3):  # start with small number
    total_loss = 0
    for images, texts in train_loader:
        images = images.to(device)
        texts = clip.tokenize(texts, truncate=True).to(device)

        # Forward
        image_features = model.encode_image(images)
        text_features = model.encode_text(texts)

        # Normalize
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        # Similarity logits
        logits_per_image = image_features @ text_features.T * 100
        logits_per_text = logits_per_image.T
        labels = torch.arange(len(images), device=device)

        # Loss
        loss_i = loss_fn(logits_per_image, labels)
        loss_t = loss_fn(logits_per_text, labels)
        loss = (loss_i + loss_t) / 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {total_loss/len(train_loader):.4f}")


Epoch 1 - Loss: nan
Epoch 2 - Loss: nan
Epoch 3 - Loss: nan
